In [1]:
%%capture
!pip3 install transformers
!pip3 install torch
!pip3 install sentencepiece
# !pip3 install tuned-lens
!pip3 install tensorrt
!pip3 install zstandard
!pip3 install datasets
!pip install wandb -qU

In [2]:
!git clone https://github.com/hunarbatra/tuned-lens

Cloning into 'tuned-lens'...
remote: Enumerating objects: 1683, done.
remote: Counting objects: 100% (625/625), done.
remote: Compressing objects: 100% (270/270), done.
remote: Total 1683 (delta 496), reused 373 (delta 353), pack-reused 1058
Receiving objects: 100% (1683/1683), 1.65 MiB | 9.31 MiB/s, done.
Resolving deltas: 100% (1079/1079), done.


In [ ]:
# !wget https://the-eye.eu/public/AI/pile/val.jsonl.zst
# !unzstd val.jsonl.zst
# !wget https://the-eye.eu/public/AI/pile/test.jsonl.zst
# !unzstd test.jsonl.zst

In [ ]:
# !mv ./test.jsonl.zst tuned-lens/test.jsonl
# !mv ./val.jsonl.zst tuned-lens/val.jsonl

In [3]:
%cd tuned-lens/

/workspace/tuned-lens


In [4]:
!pip3 install -e .

Obtaining file:///workspace/tuned-lens
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 67.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 62.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 52.0 MB/s eta 0:00:00
  Building editable for tuned-lens (pyproject.toml) ... done
  Created wheel for tuned-lens: filename=tuned_lens-0.2.0-0.editable-py3-none-any.whl size=5898 sha256=537f29b364a3ce681c2371a0f2394b815341b452c1746b98a494bb3f5ef4dab1
  Stored in directory: /tmp/pip-ephem-wheel-cache-0bkm062d/wheels/ab/e8/22/d8c19036f447ab08e2589606b226c187f26f4e6577593a1be4
Successfully 

In [5]:
!mkdir output
!mkdir ckpts

In [6]:
hf_token = '' # add HF token

In [7]:
%%writefile .env
WANDB_API_KEY= '' # WANDB_API_KEY
WANDB_ENTITY= '' # WANDB_Username
WANDB_PROJECT= '' # WANDB_PROJECT_NAME
HF_TOKEN= '' # HF_TOKEN

Writing .env


In [8]:
# Log in to your W&B account
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# from datasets import load_dataset

# dataset = load_dataset("togethercomputer/RedPajama-Data-1T-Sample")

In [ ]:
# for split, dataset in dataset.items():
#     dataset.to_json(f"squad-{split}.jsonl")

Creating json from Arrow format:   0%|          | 0/931 [00:00<?, ?ba/s]

In [9]:
import torch
from tuned_lens.nn.lenses import TunedLens
from transformers import AutoModelForCausalLM, AutoTokenizer

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# !python -m tuned_lens train \
#      --model.name gpt2 \
#      --data.name val.jsonl \
#      --per_gpu_batch_size=1

Red Pajama Run

In [10]:
!python3 -m tuned_lens train \
    --model.name meta-llama/Llama-2-7b-hf \
    --data.name togethercomputer/RedPajama-Data-1T-Sample \
    --per_gpu_batch_size=1 \
    --output './output' \
    --precision float16 \
    --split train \
    --slow_tokenizer \
    --checkpoint_freq 25 \
    --checkpoint_dir './ckpts' \
    --wandb tuned-lens-llama2-redpajama

Loading pretrained weights for 'meta-llama/Llama-2-7b-hf'...
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:04<00:00,  2.01s/it]
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Loading dataset 'togethercomputer/RedPajama-Data-1T-Sample'
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/datasets/utils/file_utils.py", line 625, in temp_file_manager
    yield f
  File "/usr/local/lib/python3.10/dist-packages/datasets/utils/file_utils.py", line 644, in get_from_cache
    http_get(
  File "/usr/local/lib/python3.10/dist-packages/datasets/utils/file_utils.py", line 421, in http_get
    temp_file.writ

In [ ]:
from google.colab import files

!zip -r tuned-lens.zip /content/tuned-lens/output /content/tuned-lens/ckpts
files.download('tuned-lens.zip')

  adding: content/tuned-lens/output/ (stored 0%)
  adding: content/tuned-lens/output/params.pt (deflated 100%)
  adding: content/tuned-lens/output/config.json (deflated 26%)
  adding: content/tuned-lens/ckpts/ (stored 0%)
  adding: content/tuned-lens/ckpts/snapshot_100.pth (deflated 92%)
  adding: content/tuned-lens/ckpts/snapshot_25.pth (deflated 13%)
  adding: content/tuned-lens/ckpts/snapshot_125.pth (deflated 92%)
  adding: content/tuned-lens/ckpts/snapshot_225.pth (deflated 92%)
  adding: content/tuned-lens/ckpts/snapshot_200.pth (deflated 92%)
  adding: content/tuned-lens/ckpts/snapshot_150.pth (deflated 92%)
  adding: content/tuned-lens/ckpts/snapshot_175.pth (deflated 92%)
  adding: content/tuned-lens/ckpts/snapshot_75.pth (deflated 92%)
  adding: content/tuned-lens/ckpts/snapshot_250.pth (deflated 92%)
  adding: content/tuned-lens/ckpts/snapshot_50.pth (deflated 92%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !mkdir -p "/content/drive/My Drive/MyModel/tuned-lens"
!cp -r "/content/tuned-lens/output" "/content/drive/My Drive/MyModel/tuned-lens"
# !cp -r "/content/tuned-lens/ckpts" "/content/drive/My Drive/MyModel/tuned-lens"

In [ ]:
import os

file_path = './output/params.pt'  # Replace with your file path
file_size = os.path.getsize(file_path)

# To get the size in bytes
print(f"Size of the file is: {file_size} bytes")

# To get the size in kilobytes (KB)
print(f"Size of the file is: {file_size / 1024:.2f} KB")

# To get the size in megabytes (MB)
print(f"Size of the file is: {file_size / (1024 ** 2):.2f} MB")

# To get the size in gigabytes (GB)
print(f"Size of the file is: {file_size / (1024 ** 3):.2f} GB")


Size of the file is: 1074019491 bytes
Size of the file is: 1048847.16 KB
Size of the file is: 1024.26 MB
Size of the file is: 1.00 GB


Pile Run

In [ ]:
!python3 -m tuned_lens train \
    --model.name meta-llama/Llama-2-7b-hf \
    --data.name val.jsonl \
    --per_gpu_batch_size=1 \
    --use_auth_token $hf_token \
    --output='.output/'

2023-08-08 01:06:43.102915: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading pretrained weights for 'meta-llama/Llama-2-7b-hf'...
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Loading checkpoint shards: 100% 2/2 [00:10<00:00,  5.42s/it]
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Loading dataset 'val.jsonl'
Extracting data files: 100% 1/1 [00:03<00:00,  3.49s/it]
Generating train split: 214670 examples [00:05, 40115.94 examples/s]
Map (num_proc=6): 100% 214670/214670 [1:25:00<00:00, 42.09 examples/s]
Using nats per token to bits per byte ratio: 0.4224037736065272
Tuned lens memory usage: 2548.52 MB in torch.float16
Gradient ac